In [1]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt

import numpy as np
#import seaborn as sns
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import accuracy_score, precision_recall_curve, precision_score, recall_score, roc_auc_score
import sys
import pandas as pd

In [ ]:
### module imports
import warnings
warnings.simplefilter('ignore')
import itertools
import numpy as np
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
import itertools
import os

### import from our files
from mliv.dgps import get_data, get_tau_fn, fn_dict
from mliv.neuralnet.utilities import log_metrics, plot_results, hyperparam_grid,\
                                     hyperparam_mult_grid, eval_performance
from mliv.neuralnet.mnist_dgps import AbstractMNISTxz
from mliv.neuralnet import AGMM,KernelLayerMMDGMM
from mliv.neuralnet.rbflayer import gaussian, inverse_multiquadric

In [ ]:
device = torch.cuda.current_device() if torch.cuda.is_available() else None

In [2]:
#ID = 15


#ind = np.unravel_index(ID-1, [3, 2, 4, 3], 'F')
ind_J = 10 #ind[0]
ind_rho = 0 #ind[1]
ind_method = 'sin'#ind[2]
ind_mode = 'NN'#ind[3]



if ind_J == 0:
    J = 2
elif ind_J == 1:
    J = 5
else:
    J = 10


if ind_rho == 0:
    rho = 0.25
else:
    rho = 0.75


if ind_method == 0:
    method = 'abs'
elif ind_method == 1:
    method = 'log'
else:
    if ind_method == 2:
        method = 'sin'
    else:
        method ='step'

if ind_mode == 0:
    mode = 'NN'
elif ind_mode ==1:
    mode = 'PPHI'
else:
    mode = 'polyPPHI'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def compute_stats(d1_train, d2_train, y_train, d1_test, d2_test, y_test, N, mode):


    print("J:",J, "rho:",rho, "method:",method, "mode:",mode)


    torch.manual_seed(2)    # reproducible

    x1 = torch.tensor(d1_train, dtype=torch.float32)
    x2 = torch.tensor(d2_train, dtype=torch.float32)
    y  = torch.tensor(y_train, dtype=torch.float32)

    # torch can only train on Variable, so convert them to Variable
    x1, x2, y = Variable(x1), Variable(x2), Variable(y)


    if mode == 'PPHI':
        net = torch.nn.Sequential(
            torch.nn.Linear(1, 1),
            torch.nn.Flatten(0, 1)
            )
    elif mode == 'polyPPHI':
        net = torch.nn.Sequential(
            torch.nn.Linear(2, 1),
            torch.nn.Flatten(0, 1)
            )
    else:
        net = torch.nn.Sequential(
            torch.nn.Linear(1, 300),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(300, 200),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(200, 1),
            )


    # print(net)  # net architecture
    optimizer = torch.optim.Adam(net.parameters(), lr=0.1)

    if mode == 'PPHI':
        p = torch.tensor([1])
        x10 = torch.unsqueeze(x1[:,0],dim=1)
        x20 = torch.unsqueeze(x2[:,0],dim=1)
        x11 = torch.unsqueeze(x1[:,1:],dim=1).pow(p)
        x21 = torch.unsqueeze(x2[:,1:],dim=1).pow(p)
    elif mode == 'polyPPHI':
        p = torch.tensor([1,2])
        x10 = torch.unsqueeze(x1[:,0],dim=1)
        x20 = torch.unsqueeze(x2[:,0],dim=1)
        x11 = torch.unsqueeze(x1[:,1:],dim=1).pow(p)
        x21 = torch.unsqueeze(x2[:,1:],dim=1).pow(p)
    else:
        x10 = torch.unsqueeze(x1[:,0],dim=1)
        x20 = torch.unsqueeze(x2[:,0],dim=1)
        x11 = torch.unsqueeze(x1[:,1:],dim=1)
        x21 = torch.unsqueeze(x2[:,1:],dim=1)


    def deepci_loss(first, second, y):
        diff1 = torch.reshape(first - second, (-1,))
        diff = diff1*(y>0) + (-diff1)*(y<0)
        loss = torch.mean(torch.minimum(torch.zeros(diff.size()),diff)**2)
        return loss


    BATCH_SIZE = 64
    EPOCH = 100
    torch_dataset = Data.TensorDataset(x10, x11, x20, x21, y)

    loader = Data.DataLoader(
        dataset=torch_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=True, num_workers=2,)
    iteration = 0

    # start training
    for epoch in range(EPOCH):
        for step, (batch_x10, batch_x11, batch_x20, batch_x21, batch_y) in enumerate(loader): # for each training step

            b_x10 = Variable(batch_x10)
            b_x11 = Variable(batch_x11)
            b_x20 = Variable(batch_x20)
            b_x21 = Variable(batch_x21)
            b_y = Variable(batch_y)

            prediction_1 = 0.01*b_x10 + net(b_x11)     # input x and predict based on x
            prediction_2 = 0.01*b_x20 + net(b_x21)

            loss = deepci_loss(prediction_1, prediction_2, b_y)     # must be (1. nn output, 2. target)
            #if iteration%500 == 0:
            #    print(loss)
            iteration +=1    
            optimizer.zero_grad()   # clear gradients for next train
            loss.backward()         # backpropagation, compute gradients
            optimizer.step()        # apply gradients


    x1_test = torch.tensor(d1_test, dtype=torch.float32)
    x2_test = torch.tensor(d2_test, dtype=torch.float32)
    y       = torch.tensor(y_test, dtype=torch.float32)


    if mode == 'PPHI':
        x10_test = torch.unsqueeze(x1_test[:,0],dim=1)
        x20_test = torch.unsqueeze(x2_test[:,0],dim=1)
        x11_test = torch.unsqueeze(x1_test[:,1:],dim=1).pow(p)
        x21_test = torch.unsqueeze(x2_test[:,1:],dim=1).pow(p)
    elif mode == 'polyPPHI':
        x10_test = torch.unsqueeze(x1_test[:,0],dim=1)
        x20_test = torch.unsqueeze(x2_test[:,0],dim=1)
        x11_test = torch.unsqueeze(x1_test[:,1:],dim=1).pow(p)
        x21_test = torch.unsqueeze(x2_test[:,1:],dim=1).pow(p)
    else:
        x10_test = torch.unsqueeze(x1_test[:,0],dim=1)
        x20_test = torch.unsqueeze(x2_test[:,0],dim=1)
        x11_test = torch.unsqueeze(x1_test[:,1:],dim=1)
        x21_test = torch.unsqueeze(x2_test[:,1:],dim=1)


    if mode == 'NN':
        y_pred = 2*(0.01*x10_test.flatten() + net(x11_test).data.numpy().flatten() > \
                0.01*x20_test.flatten() + net(x21_test).data.numpy().flatten())-1
    else:
        y_pred = 2*((0.01*x10_test + net(x11_test)).flatten() > \
                (0.01*x20_test + net(x21_test)).flatten())-1

    f1 = f1_score(y.numpy(),y_pred.numpy())
        
    return f1

In [ ]:
def simulateData(N, J, K, func, rho, sim):
    np.random.seed(sim)
    D = np.random.uniform(-10,10,(N,J + 1,K))
    x1 = np.array([])
    x2 = np.array([])
    y  = np.array([])
    for n in range(N):
        err = np.random.normal(0,0.01) # epsilon_m
        D[n,1:,1] = D[n,1:,1] + rho*err
        D[n,1:,0] = D[n,1:,0] 
        choice = np.argmax(0.01*D[n,:,0] + func(D[n,:,1:]) + np.random.normal(0,0.1,(1,J + 1)) + rho*err) # x_0+f(x_1) for each product
        D[n,1:,0] = D[n,1:,0]  - rho*err # why?
        if choice == 0:
            continue
        for j in range(1, J+1):
            if j == choice:
                continue
            else:
                #print(D[n,choice,:])
                if x1.size == 0:
                    x1 = D[n,choice,:]
                    x2 = D[n,j,:]
                    y  = np.array([1])
                else:    
                    x1 = np.c_[x1, D[n,choice,:]]
                    x2 = np.c_[x2, D[n,j,:]]
                    y  = np.c_[y,np.array([1])]
    return x1.T, x2.T, y.T


print(method)

def get_function(func_str):
    if func_str == "abs":
        return (lambda x: np.abs(x@np.ones((1,1))).flatten(), 
                lambda x: torch.abs(x@np.ones((1,1))).flatten())
    elif func_str == "log":
        return (lambda x: np.log(np.abs(x@np.ones((1,1)))).flatten(), 
                lambda x: torch.log(torch.abs(x@np.ones((1,1)))).flatten())
    elif func_str == "sin":
        return (lambda x: np.sin(x@np.ones((1,1))).flatten(), 
                lambda x: torch.sin(x@np.ones((1,1))).flatten())
    else:
        return (lambda x: np.sign(np.abs(np.abs(x)-5)-2)@np.ones((1,1)).flatten(), 
                lambda x: torch.sign(torch.abs(torch.abs(x)-5)-2)@np.ones((1,1)).flatten())

In [ ]:
def runConfig(J, rho, method, mode):
    N = 2000
    K = 2
    a, b = get_function(method)
    f1s = []
    for i in range(1,2): 
        d1_train, d2_train, y_train = simulateData(N, J, K, a, rho, i)
        d1_test,  d2_test,  y_test  = simulateData(N, J, K, a, rho, i+1)
        f1  = compute_stats(d1_train, d2_train, y_train, d1_test, d2_test, y_test, d1_train.shape[0], mode)
        print(f1)
        f1s.append(f1)
    return np.mean(f1s)

result = runConfig(J, rho, method, mode)

In [ ]:
print(result)

In [ ]:
with open("result-single.txt", "a+") as text_file:
    text_file.write("J: %s, rho: %s, method: %s, mode: %s, result: %s \n" %(J, rho, method, mode, result))


result = pd.DataFrame([result])
filename = str(ID)+".csv"
result.to_csv(filename,index=False,header=False)